In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import graphviz
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

from sklearn.preprocessing import LabelEncoder

In [19]:
data_raw = pd.read_csv('./csvfiles/통신고객이탈.csv')

In [20]:
data = data_raw.copy()

In [21]:
data.tail(10)

,CHURN,CUSTOMER_ID,GENDER,AGE,CHARGE_TYPE,HANDSET,USAGE_BAND,SERVICE_DURATION,DROPPED_CALLS,PEAK_CALLS_NO,PEAK_CALLS_TIME,WEEKEND_CALLS_NO,WEEKEND_CALLS_TIME,TOTAL_CALLS_NO,TOTAL_CALLS_TIME
920,Churned,K406200,F,27,CAT 200,S50,MedHigh,9.63,15,302,1201.8,32,83.4,431,1482.3
921,Churned,K406570,M,21,CAT 200,BS110,Med,42.50,14,257,957.6,1,9.0,322,1149.0
922,Churned,K406950,M,37,CAT 200,BS110,Med,47.80,12,163,1181.4,5,45.5,180,1247.9
923,Churned,K409630,F,26,CAT 200,SOP10,High,35.53,13,381,1768.2,8,44.4,645,2334.0
924,Churned,K412310,F,14,Play 100,SOP20,Med,14.37,14,118,199.2,5,15.0,140,552.9
925,Churned,K413330,M,19,Play 100,SOP10,Med,17.40,12,60,100.8,27,59.4,127,628.2
926,Churned,K413410,F,59,Play 100,SOP20,Med,17.37,15,82,130.2,12,25.8,357,650.7
927,Churned,K415490,F,33,Play 300,S50,Med,44.93,12,174,286.2,25,67.0,596,1039.0
928,Churned,K415880,F,25,Play 300,S50,Med,40.43,14,315,552.0,0,0.0,609,1188.3
929,Churned,K416650,F,30,Play 300,SOP10,Med,31.47,12,16,306.6,37,109.8,367,957.0


In [22]:
z = {'Active':1, 'Churned':0}
data['CHURN']=data['CHURN'].map(z)

In [23]:
data.head()

,CHURN,CUSTOMER_ID,GENDER,AGE,CHARGE_TYPE,HANDSET,USAGE_BAND,SERVICE_DURATION,DROPPED_CALLS,PEAK_CALLS_NO,PEAK_CALLS_TIME,WEEKEND_CALLS_NO,WEEKEND_CALLS_TIME,TOTAL_CALLS_NO,TOTAL_CALLS_TIME
0,1,K102990,F,31,CAT 100,ASAD170,Med,33.33,6,218,379.8,9,21.6,366,632.4
1,1,K103280,M,27,CAT 100,S50,Med,30.87,10,373,656.4,11,28.0,411,810.4
2,1,K103660,M,27,CAT 100,ASAD170,Med,49.40,11,260,582.0,0,0.0,265,614.1
3,1,K103730,M,61,CAT 100,ASAD170,Med,48.60,1,294,661.8,14,101.5,326,844.0
4,1,K104560,F,52,CAT 100,S50,Med,56.03,0,221,555.6,7,25.0,274,673.0


In [24]:
gender = {'M':1, 'F':0}
data["GENDER"] = data["GENDER"].map(gender)

In [25]:
data.head()

,CHURN,CUSTOMER_ID,GENDER,AGE,CHARGE_TYPE,HANDSET,USAGE_BAND,SERVICE_DURATION,DROPPED_CALLS,PEAK_CALLS_NO,PEAK_CALLS_TIME,WEEKEND_CALLS_NO,WEEKEND_CALLS_TIME,TOTAL_CALLS_NO,TOTAL_CALLS_TIME
0,1,K102990,0,31,CAT 100,ASAD170,Med,33.33,6,218,379.8,9,21.6,366,632.4
1,1,K103280,1,27,CAT 100,S50,Med,30.87,10,373,656.4,11,28.0,411,810.4
2,1,K103660,1,27,CAT 100,ASAD170,Med,49.40,11,260,582.0,0,0.0,265,614.1
3,1,K103730,1,61,CAT 100,ASAD170,Med,48.60,1,294,661.8,14,101.5,326,844.0
4,1,K104560,0,52,CAT 100,S50,Med,56.03,0,221,555.6,7,25.0,274,673.0


In [26]:
data.isna().any()

CHURN                 False
CUSTOMER_ID           False
GENDER                False
AGE                   False
CHARGE_TYPE           False
HANDSET               False
USAGE_BAND            False
SERVICE_DURATION      False
DROPPED_CALLS         False
PEAK_CALLS_NO         False
PEAK_CALLS_TIME       False
WEEKEND_CALLS_NO      False
WEEKEND_CALLS_TIME    False
TOTAL_CALLS_NO        False
TOTAL_CALLS_TIME      False
dtype: bool

# Model

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 15 columns):
CHURN                 930 non-null int64
CUSTOMER_ID           930 non-null object
GENDER                930 non-null int64
AGE                   930 non-null int64
CHARGE_TYPE           930 non-null object
HANDSET               930 non-null object
USAGE_BAND            930 non-null object
SERVICE_DURATION      930 non-null float64
DROPPED_CALLS         930 non-null int64
PEAK_CALLS_NO         930 non-null int64
PEAK_CALLS_TIME       930 non-null float64
WEEKEND_CALLS_NO      930 non-null int64
WEEKEND_CALLS_TIME    930 non-null float64
TOTAL_CALLS_NO        930 non-null int64
TOTAL_CALLS_TIME      930 non-null float64
dtypes: float64(4), int64(7), object(4)
memory usage: 109.1+ KB


In [28]:
object_cols = data.select_dtypes(include=['object']).columns.tolist()

In [30]:
object_cols

['CUSTOMER_ID', 'CHARGE_TYPE', 'HANDSET', 'USAGE_BAND']

In [31]:
from sklearn.preprocessing import LabelEncoder
def MultiLabelEncoder(columnlist,dataframe):
    for i in columnlist:

        labelencoder_X=LabelEncoder()
        dataframe[i]=labelencoder_X.fit_transform(dataframe[i])

In [32]:
MultiLabelEncoder(object_cols, data)

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 15 columns):
CHURN                 930 non-null int64
CUSTOMER_ID           930 non-null int64
GENDER                930 non-null int64
AGE                   930 non-null int64
CHARGE_TYPE           930 non-null int64
HANDSET               930 non-null int64
USAGE_BAND            930 non-null int64
SERVICE_DURATION      930 non-null float64
DROPPED_CALLS         930 non-null int64
PEAK_CALLS_NO         930 non-null int64
PEAK_CALLS_TIME       930 non-null float64
WEEKEND_CALLS_NO      930 non-null int64
WEEKEND_CALLS_TIME    930 non-null float64
TOTAL_CALLS_NO        930 non-null int64
TOTAL_CALLS_TIME      930 non-null float64
dtypes: float64(4), int64(11)
memory usage: 109.1 KB


In [34]:
from sklearn.model_selection import train_test_split

y = data["CHURN"]
x = data.drop("CHURN", axis = 1)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=42)
print("train data X size:", train_x.shape)
print("train data y size:", train_y.shape)
print("test data X size:", test_x.shape)
print("test data y size:", test_y.shape)


tree_uncustom = DecisionTreeClassifier(random_state=1234, )
tree_uncustom.fit(train_x, train_y)
# 훈련 데이터 정확도
print("Accuracy on training set:{:.3f}".format(tree_uncustom.score(train_x, train_y)))

# test 데이터 정확도
print("Accuracy on test set:{:.3f}".format(tree_uncustom.score(test_x, test_y)))

train data X size: (651, 14)
train data y size: (651,)
test data X size: (279, 14)
test data y size: (279,)
Accuracy on training set:1.000
Accuracy on test set:0.907


In [38]:
from sklearn.model_selection import GridSearchCV

criterion = ['gini', 'entropy']
max_dapth = list(range(1,31))
list_min_leaf_size = [i * 3 for i in range (1,6)]

parameters = {'criterion':criterion, 'max_depth': max_dapth,
             'min_samples_leaf':list_min_leaf_size}

dt_random = GridSearchCV(estimator = tree_uncustom, param_grid = parameters, 
                               cv = 10, n_jobs=-1)
dt_random.fit(train_x, train_y)

params = dt_random.best_params_

print(params)

print("Accuracy on test set after searching parameters:{:.3f}".format(dt_random.score(test_x, test_y)))

{'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 9}
Accuracy on test set after searching parameters:0.932


### 